In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

In [2]:
#每个页面抓取评论数方式相同,应建立评论数抽取函数
#找到返回了评论数的url模板,对比发现标识文章的是&newsid,相同部分为comos-,其他使用{}参数替换
#这里的channel也是参数,不过暂时爬取gn(国内新闻评论)
commentURL='http://comment5.news.sina.com.cn/page/info?version=1&format=js\
&channel=gn&newsid=comos-{0}&group=0&compress=1&ie=utf-8&oe=utf-8&page=1\
&show_reply=1&page_size=20'

def getCommentCount(newsurl):
    #先从newsurl内取得newsid
    m=re.search('doc-i(.+).shtml',newsurl)
    newsid=m.group(1)
    print(newsid)
    #将newsid放到请求模板中,发送请求
    resinfo=requests.get(commentURL.format(newsid))
    resinfo.encoding='utf-8'
    #将得到的json转化为字典
    jd = json.loads(resinfo.text.strip('var data='))
    #得到内嵌字典里的评论数
    return jd['result']['count']['total']

In [3]:
#现在可以直接使用函数爬取该站点channel=gn的评论数了
#如果是科技类就要另外做判断
news='http://tech.sina.com.cn/csj/2018-01-17/doc-ifyqqieu7096578.shtml'
newsurl='http://news.sina.com.cn/china/xlxw/2018-01-17/doc-ifyqrewi9270282.shtml'
print(getCommentCount(newsurl))
print(getCommentCount(news))

fyqrewi9270282
3905
fyqqieu7096578


KeyError: 'count'

In [4]:
#抓取新闻函数
def getNewsDetail(newsurl):
    result={}
    res = requests.get(newsurl)
    res.encoding='utf-8'
    soup=BeautifulSoup(res.text,'html.parser')
    result['title']=soup.select('.main-title')[0].text
    dt=datetime.strptime(soup.select('.date-source')[0].contents[1].text,'%Y年%m月%d日 %H:%M')
    result['time']=datetime.strftime(dt,'%Y-%m-%d')
    result['newssource']=soup.select('.date-source a')[0].text
#     result['article']=[]
#     for p in soup.select('.article p')[:-1]:
#         result['article'].append(p.text.strip())
#     result['article']='\n\n'.join( result['article'])
    result['article']='\n\n'.join([p.text.strip() for p in soup.select('.article p')[:-1]])
    result['editor']=soup.select('.show_author')[0].text.strip('责任编辑：').strip()
    result['comments']=getCommentCount(newsurl)
    return result

In [5]:
newsurl='http://news.sina.com.cn/china/xlxw/2018-01-17/doc-ifyqrewi9270282.shtml'
Detail=getNewsDetail(newsurl)
print (Detail)

fyqrewi9270282
{'title': '习近平亲民外交六个瞬间', 'time': '2018-01-17', 'newssource': '央视新闻', 'article': '原标题：习近平亲民外交六个瞬间\n\n作为一国元首，常在镜头前出现的习近平很少有机会展现他个人性情的一面。但当他与多年未谋面的老友重聚时，或在繁忙的外事活动间隙里，人们看到了一个不一样的习近平。\n\nAs the busy leader of a country of over 1.3 billion people， Chinese President Xi Jinping rarely has a chance to show his personal side to the public。 But when revisiting old friends or taking a break from diplomatic tasks， he has revealed glimpses of his personality to the world。\n\n三十年的友谊，你看看\n\nIn 1985， the Dvorchaks hosted Xi during his visit to Iowa。 Xi invited them to China 30 years later\xa0and hosted them at the Diaoyutai State Guesthouse。\n\n我过去，您别站起来了\n\nXi awarded medals to Russian veterans for their contributions to China’s victory against Japanese aggression and the World Anti-Fascist War。 Many of the veterans attending the event had limited mobility， so Xi brought the medals over to them。\n\n我也是了却一个心愿\n\nIn 2001， Jim Bacon， then premier of Tasmania， met Xi in China and invited hi